# Delta table 
- Update 
- Delete

In [2]:
import os
import logging
import configparser

# Read from config

In [3]:
config = configparser.ConfigParser()
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
config.read(os.path.join(root_path, "configs","config.ini"))

['/opt/spark/work-dir/configs/config.ini']

# Logging function 
- 

In [4]:
# Create a logger
logger = logging.getLogger("EMP_DELTA_CRUD")

# Set the level of the logger
logger.setLevel(logging.INFO)

# Create a FileHandler object and specify the file path
log_file_path = config['LOGS']['log_path']
file_path = os.path.join(log_file_path,"employee_delta_crud.log")
os.makedirs(os.path.dirname(file_path), exist_ok=True)
file_handler = logging.FileHandler(file_path)

# Set the format for the log messages
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Logger handler 
logger.addHandler(file_handler)

# Read from delta table

In [5]:
try:
    logger.info("start reading the delta table")
    emp_delta_table = os.path.join(config['DELTA']['delta_table_path'],"datq", "employee")
    empdf = (spark
             .read
             .format("delta")
             .load(emp_delta_table)
            )
    logger.info("end reading the delta")
except: 
    logger.error("cannot read delta table")

In [6]:
empdf.show()

23/11/22 16:52:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|         No|                        0|         0|
|Bachelors|       2013|     Pune|          1| 28|Female|         No|                        3|         1|
|Bachelors|       2014|New Delhi|          3| 38|Female|         No|                        2|         0|
|  Masters|       2016|Bangalore|          3| 27|  Male|         No|                        5|         1|
|  Masters|       2017|     Pune|          3| 24|  Male|        Yes|                        2|         1|
|Bachelors|       2016|Bangalore|          3| 22|  Male|         No|                        0|         0|
|Bachelors|       2015|New Delhi|          3| 